## Connecting to tables


Need to have a driver to connect to tables in a database.
- sqlite3 --> Sqlite
- psycopg2 --> Postgres/Reshift
- mariadb --> Mariadb
- pyodbc --> SQL Server
- cx_Oracle --> Oracle

In [1]:
import pandas as pd
import numpy as np
import sqlite3 as sqlite

In [2]:
df_url = 'https://raw.githubusercontent.com/allisonhorst/palmerpenguins/master/inst/extdata/penguins_raw.csv'

penguins_df = pd.read_csv(df_url)

penguins_df.head()

,studyName,Sample Number,Species,Region,Island,Stage,Individual ID,Clutch Completion,Date Egg,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo),Comments
0,PAL0708,1,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A1,Yes,2007-11-11,39.1,18.7,181.0,3750.0,MALE,NaN,NaN,Not enough blood for isotopes.
1,PAL0708,2,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A2,Yes,2007-11-11,39.5,17.4,186.0,3800.0,FEMALE,8.94956,-24.69454,NaN
2,PAL0708,3,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N2A1,Yes,2007-11-16,40.3,18.0,195.0,3250.0,FEMALE,8.36821,-25.33302,NaN
3,PAL0708,4,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N2A2,Yes,2007-11-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Adult not sampled.
4,PAL0708,5,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N3A1,Yes,2007-11-16,36.7,19.3,193.0,3450.0,FEMALE,8.76651,-25.32426,NaN


In [3]:
def fix_col(name):
    '''
        Function to clean up the column names
    '''
    # Find the begining of a parenthesis
    par_index = name.find('(')
    # If there is no parenthesis
    if par_index == -1:
        # Keep the length of the name
        par_index = len(name)
        
    else:
        # If there is a parenthesis, select till before the parenthesis
        par_index -= 1
        
    return name.lower().replace(' ', '_').replace('#', 'num').replace('%', 'per')[:par_index]

In [4]:
# Create your Database
con = sqlite.connect('data/penguins.db')

# Create a Cursor
cur = con.cursor()

# Drop penguin table if it exists
cur.execute('DROP TABLE IF EXISTS penguin')

# Close the Cursor
cur.close()



df = (penguins_df
        .rename(columns=fix_col)
        .assign(date_egg=lambda df_: pd.to_datetime(df_['date_egg']))
)

# Load the dataframe into the penguin db
df.to_sql('penguin', con)

con.close()

In [5]:
def run_sql(sql_string, conn_str, values=None, size=20):
    # Establsh a connection to the database
    con = sqlite.connect(conn_str)
    # Get a cursor object
    cur = con.cursor()
    if values is not None:
        res = cur.execute(sql_string, values)
        
    else:
        res  = cur.execute(sql_string)
        
    df = pd.DataFrame(res.fetchmany(size), columns=[d[0] for d in res.description])
    
    # Close the cursor
    cur.close()
    # Close the connection
    con.close()
    
    return df

## Selecting DATA

Conventions
- Field (column names) should be lowercased, use undersore to represent ' '
- Field (column names) should be singular
- Reserved words/SQL keywords should be uppercased

In [6]:
con_str = 'data/penguins.db'

run_sql('''
        SELECT *
        FROM penguin  
        ''', conn_str=con_str)

,index,studyname,sample_number,species,region,island,stage,individual_id,clutch_completion,date_egg,culmen_length,culmen_depth,flipper_length,body_mass,sex,delta_15_n,delta_13_c,comments
0,0,PAL0708,1,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A1,Yes,2007-11-11 00:00:00,39.1,18.7,181.0,3750.0,MALE,NaN,NaN,Not enough blood for isotopes.
1,1,PAL0708,2,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A2,Yes,2007-11-11 00:00:00,39.5,17.4,186.0,3800.0,FEMALE,8.94956,-24.69454,None
2,2,PAL0708,3,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N2A1,Yes,2007-11-16 00:00:00,40.3,18.0,195.0,3250.0,FEMALE,8.36821,-25.33302,None
3,3,PAL0708,4,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N2A2,Yes,2007-11-16 00:00:00,NaN,NaN,NaN,NaN,None,NaN,NaN,Adult not sampled.
4,4,PAL0708,5,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N3A1,Yes,2007-11-16 00:00:00,36.7,19.3,193.0,3450.0,FEMALE,8.76651,-25.32426,None
5,5,PAL0708,6,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N3A2,Yes,2007-11-16 00:00:00,39.3,20.6,190.0,3650.0,MALE,8.66496,-25.29805,None
6,6,PAL0708,7,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N4A1,No,2007-11-15 00:00:00,38.9,17.8,181.0,3625.0,FEMALE,9.18718,-25.21799,Nest never observed with full clutch.
7,7,PAL0708,8,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N4A2,No,2007-11-15 00:00:00,39.2,19.6,195.0,4675.0,MALE,9.46060,-24.89958,Nest never observed with full clutch.
8,8,PAL0708,9,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N5A1,Yes,2007-11-09 00:00:00,34.1,18.1,193.0,3475.0,None,NaN,NaN,No blood sample obtained.
9,9,PAL0708,10,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N5A2,Yes,2007-11-09 00:00:00,42.0,20.2,190.0,4250.0,None,9.13362,-25.09368,No blood sample obtained for sexing.


In [7]:
# Specifying columns to select with aliasing (optional)

run_sql('''
        SELECT p.studyname,
                p.species,
                p.date_egg,
                p.culmen_length,
                p.culmen_depth,
                p.sex
        FROM penguin as p
        ''', conn_str=con_str)



,studyname,species,date_egg,culmen_length,culmen_depth,sex
0,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-11 00:00:00,39.1,18.7,MALE
1,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-11 00:00:00,39.5,17.4,FEMALE
2,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-16 00:00:00,40.3,18.0,FEMALE
3,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-16 00:00:00,NaN,NaN,None
4,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-16 00:00:00,36.7,19.3,FEMALE
5,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-16 00:00:00,39.3,20.6,MALE
6,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-15 00:00:00,38.9,17.8,FEMALE
7,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-15 00:00:00,39.2,19.6,MALE
8,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-09 00:00:00,34.1,18.1,None
9,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-09 00:00:00,42.0,20.2,None


In [8]:
# Specifying columns to select with aliasing (optional)

run_sql('''
        SELECT p.studyname as name,
                p.species as species,
                p.date_egg as date,
                p.culmen_length,
                p.culmen_depth,
                p.sex
        FROM penguin as p
        ''', conn_str=con_str)

,name,species,date,culmen_length,culmen_depth,sex
0,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-11 00:00:00,39.1,18.7,MALE
1,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-11 00:00:00,39.5,17.4,FEMALE
2,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-16 00:00:00,40.3,18.0,FEMALE
3,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-16 00:00:00,NaN,NaN,None
4,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-16 00:00:00,36.7,19.3,FEMALE
5,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-16 00:00:00,39.3,20.6,MALE
6,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-15 00:00:00,38.9,17.8,FEMALE
7,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-15 00:00:00,39.2,19.6,MALE
8,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-09 00:00:00,34.1,18.1,None
9,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-09 00:00:00,42.0,20.2,None


In [9]:
# Specifying columns to select with aliasing (optional) and LIMIT

run_sql('''
        SELECT  p.studyname as name,
                p.species as species,
                p.date_egg as date,
                p.culmen_length,
                p.culmen_depth,
                p.sex
        FROM penguin as p
        LIMIT 10
        ''', conn_str=con_str)

,name,species,date,culmen_length,culmen_depth,sex
0,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-11 00:00:00,39.1,18.7,MALE
1,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-11 00:00:00,39.5,17.4,FEMALE
2,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-16 00:00:00,40.3,18.0,FEMALE
3,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-16 00:00:00,NaN,NaN,None
4,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-16 00:00:00,36.7,19.3,FEMALE
5,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-16 00:00:00,39.3,20.6,MALE
6,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-15 00:00:00,38.9,17.8,FEMALE
7,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-15 00:00:00,39.2,19.6,MALE
8,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-09 00:00:00,34.1,18.1,None
9,PAL0708,Adelie Penguin (Pygoscelis adeliae),2007-11-09 00:00:00,42.0,20.2,None


Order of writing SQL:
* SELECT... - select columns, aggregations
* FROM...   - Tables to query and join 
* WHERE...  - Filter Selection
* GROUP BY  - Columns to group for aggregations
* HAVING... - Filter on Aggregations   
* ORDER BY  - Order results
* LIMIT...  - Limit rows returned
* OFFSET... - Offset results


Order of execution:
* FROM
* WHERE
* GROUP BY
* HAVING
* SELECT
* ORDER BY
* LIMIT
* OFFSET

## Counting (with DISTINCT)



In [10]:
# Count is a function in SQL
# count(*) counts every record in the table including NULLs

run_sql('''
        SELECT count(*) as count_of_rows
        FROM penguin
        ''', conn_str=con_str)

,count_of_rows
0,344


In [11]:
# count(culmen_length) counts only the NON-NULL values (i.e. ignore nulls)

run_sql('''
        SELECT count(culmen_length) as count_of_culmen
        FROM penguin
        ''', conn_str=con_str)

,count_of_culmen
0,342


In [12]:
# Count the nulls in culmen_length
run_sql('''
        SELECT count(*) - count(culmen_length) as culmen_null_size
        FROM penguin
        ''', conn_str=con_str)

,culmen_null_size
0,2


In [13]:
# Count the nulls in culmen_length
# CASE WHEN culmen_length IS NULL THEN 1 ELSE 0 END
run_sql('''
        SELECT sum(CASE WHEN culmen_length IS NULL THEN 1 END) as culmen_null_size
        FROM penguin
        ''', conn_str=con_str)

,culmen_null_size
0,2


In [14]:
# Show the DISTINCT records
run_sql('''
        SELECT DISTINCT *
        FROM penguin
        ''', 
        conn_str=con_str)

,index,studyname,sample_number,species,region,island,stage,individual_id,clutch_completion,date_egg,culmen_length,culmen_depth,flipper_length,body_mass,sex,delta_15_n,delta_13_c,comments
0,0,PAL0708,1,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A1,Yes,2007-11-11 00:00:00,39.1,18.7,181.0,3750.0,MALE,NaN,NaN,Not enough blood for isotopes.
1,1,PAL0708,2,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A2,Yes,2007-11-11 00:00:00,39.5,17.4,186.0,3800.0,FEMALE,8.94956,-24.69454,None
2,2,PAL0708,3,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N2A1,Yes,2007-11-16 00:00:00,40.3,18.0,195.0,3250.0,FEMALE,8.36821,-25.33302,None
3,3,PAL0708,4,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N2A2,Yes,2007-11-16 00:00:00,NaN,NaN,NaN,NaN,None,NaN,NaN,Adult not sampled.
4,4,PAL0708,5,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N3A1,Yes,2007-11-16 00:00:00,36.7,19.3,193.0,3450.0,FEMALE,8.76651,-25.32426,None
5,5,PAL0708,6,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N3A2,Yes,2007-11-16 00:00:00,39.3,20.6,190.0,3650.0,MALE,8.66496,-25.29805,None
6,6,PAL0708,7,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N4A1,No,2007-11-15 00:00:00,38.9,17.8,181.0,3625.0,FEMALE,9.18718,-25.21799,Nest never observed with full clutch.
7,7,PAL0708,8,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N4A2,No,2007-11-15 00:00:00,39.2,19.6,195.0,4675.0,MALE,9.46060,-24.89958,Nest never observed with full clutch.
8,8,PAL0708,9,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N5A1,Yes,2007-11-09 00:00:00,34.1,18.1,193.0,3475.0,None,NaN,NaN,No blood sample obtained.
9,9,PAL0708,10,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N5A2,Yes,2007-11-09 00:00:00,42.0,20.2,190.0,4250.0,None,9.13362,-25.09368,No blood sample obtained for sexing.


In [15]:
# Select distinct species
run_sql('''
        SELECT DISTINCT species
        FROM penguin
        ''', 
        conn_str=con_str)

,species
0,Adelie Penguin (Pygoscelis adeliae)
1,Gentoo penguin (Pygoscelis papua)
2,Chinstrap penguin (Pygoscelis antarctica)


In [16]:
# Count distinct species
run_sql('''
        SELECT count(DISTINCT species) as count_of_distinct_species
        FROM penguin
        ''', 
        conn_str=con_str)

,count_of_distinct_species
0,3


In [17]:
# Count distinct species
run_sql('''
        SELECT count(DISTINCT species) as count_of_distinct_species,
               count(DISTINCT region) as count_of_distinct_region,
               count(DISTINCT date_egg) as count_of_distinct_dates
        FROM penguin
        ''', 
        conn_str=con_str)

,count_of_distinct_species,count_of_distinct_region,count_of_distinct_dates
0,3,1,50


## Filtering

In [18]:
# Selecting specific entries
run_sql('''
        SELECT  culmen_length,
                culmen_depth,
                flipper_length,
                body_mass
        FROM penguin
        WHERE culmen_length < 35
        ''',
        conn_str=con_str
        )

,culmen_length,culmen_depth,flipper_length,body_mass
0,34.1,18.1,193.0,3475.0
1,34.6,21.1,198.0,4400.0
2,34.4,18.4,184.0,3325.0
3,34.5,18.1,187.0,2900.0
4,33.5,19.0,190.0,3600.0
5,34.6,17.2,189.0,3200.0
6,34.0,17.1,185.0,3400.0
7,33.1,16.1,178.0,2900.0
8,32.1,15.5,188.0,3050.0


In [19]:
# Combine logic with AND, OR
run_sql('''
        SELECT  culmen_length,
                culmen_depth,
                flipper_length,
                body_mass
        FROM penguin
        WHERE culmen_length < 35 AND culmen_depth < 20
        ''',
        conn_str=con_str
        )

,culmen_length,culmen_depth,flipper_length,body_mass
0,34.1,18.1,193.0,3475.0
1,34.4,18.4,184.0,3325.0
2,34.5,18.1,187.0,2900.0
3,33.5,19.0,190.0,3600.0
4,34.6,17.2,189.0,3200.0
5,34.0,17.1,185.0,3400.0
6,33.1,16.1,178.0,2900.0
7,32.1,15.5,188.0,3050.0


In [20]:
# Combine logic with AND, OR
run_sql('''
        SELECT  culmen_length,
                culmen_depth,
                culmen_length/culmen_depth as length_to_depth
        FROM penguin
        WHERE length_to_depth > 0
        ORDER BY length_to_depth DESC
        ''',
        conn_str=con_str
        )

,culmen_length,culmen_depth,length_to_depth
0,51.3,14.2,3.612676
1,50.2,14.3,3.510490
2,59.6,17.0,3.505882
3,46.1,13.2,3.492424
4,54.3,15.7,3.458599
5,48.7,14.1,3.453901
6,47.2,13.7,3.445255
7,46.5,13.5,3.444444
8,55.1,16.0,3.443750
9,48.5,14.1,3.439716


In [21]:
# Combine logic with AND, OR
run_sql('''
        SELECT  culmen_length,
                culmen_depth,
                culmen_length/culmen_depth as length_to_depth
        FROM penguin
        WHERE length_to_depth > 0 and culmen_length < 40
        ORDER BY length_to_depth DESC
        ''',
        conn_str=con_str
        )

,culmen_length,culmen_depth,length_to_depth
0,39.5,16.7,2.365269
1,37.7,16.0,2.356250
2,38.1,16.5,2.309091
3,39.6,17.2,2.302326
4,39.0,17.1,2.280702
5,38.6,17.0,2.270588
6,39.5,17.4,2.270115
7,38.8,17.2,2.255814
8,36.2,16.1,2.248447
9,38.6,17.2,2.244186


In [22]:
# Combine logic with BETWEEN (includes the interval)
# This is used for DATES
run_sql('''
        SELECT  culmen_length,
                culmen_depth,
                culmen_length/culmen_depth as length_to_depth
        FROM penguin
        WHERE length_to_depth BETWEEN 2 AND 3
        ORDER BY length_to_depth DESC
        ''',
        conn_str=con_str
        )

,culmen_length,culmen_depth,length_to_depth
0,40.9,13.7,2.985401
1,43.2,14.5,2.979310
2,46.4,15.6,2.974359
3,48.4,16.3,2.969325
4,49.8,16.8,2.964286
5,50.8,17.3,2.936416
6,48.1,16.4,2.932927
7,46.3,15.8,2.930380
8,45.0,15.4,2.922078
9,46.8,16.1,2.906832


In [23]:
# Combine logic with IN (includes the interval)
run_sql('''
        SELECT  species,
                culmen_length,
                culmen_depth,
                culmen_length/culmen_depth as length_to_depth
        FROM penguin
        WHERE species in ('Adelie Penguin (Pygoscelis adeliae)', 'Chinstrap penguin (Pygoscelis antarctica)')
        ORDER BY length_to_depth DESC
        ''',
        conn_str=con_str
        )

,species,culmen_length,culmen_depth,length_to_depth
0,Chinstrap penguin (Pygoscelis antarctica),58.0,17.8,3.258427
1,Chinstrap penguin (Pygoscelis antarctica),48.1,16.4,2.932927
2,Chinstrap penguin (Pygoscelis antarctica),49.8,17.3,2.878613
3,Chinstrap penguin (Pygoscelis antarctica),52.0,18.1,2.872928
4,Chinstrap penguin (Pygoscelis antarctica),50.9,17.9,2.843575
5,Chinstrap penguin (Pygoscelis antarctica),46.8,16.5,2.836364
6,Chinstrap penguin (Pygoscelis antarctica),47.5,16.8,2.827381
7,Chinstrap penguin (Pygoscelis antarctica),46.9,16.6,2.825301
8,Chinstrap penguin (Pygoscelis antarctica),51.3,18.2,2.818681
9,Chinstrap penguin (Pygoscelis antarctica),55.8,19.8,2.818182


## Working with DATES


In [24]:
# Select Month
# Function might vary depending on database (date_part/extract for Postgres)
# see https://strftime.org
run_sql('''
        SELECT  date_egg as date,
                strftime('%m', date_egg) as month,
                species,
                culmen_length
        FROM penguin
        ''', conn_str=con_str)

,date,month,species,culmen_length
0,2007-11-11 00:00:00,11,Adelie Penguin (Pygoscelis adeliae),39.1
1,2007-11-11 00:00:00,11,Adelie Penguin (Pygoscelis adeliae),39.5
2,2007-11-16 00:00:00,11,Adelie Penguin (Pygoscelis adeliae),40.3
3,2007-11-16 00:00:00,11,Adelie Penguin (Pygoscelis adeliae),NaN
4,2007-11-16 00:00:00,11,Adelie Penguin (Pygoscelis adeliae),36.7
5,2007-11-16 00:00:00,11,Adelie Penguin (Pygoscelis adeliae),39.3
6,2007-11-15 00:00:00,11,Adelie Penguin (Pygoscelis adeliae),38.9
7,2007-11-15 00:00:00,11,Adelie Penguin (Pygoscelis adeliae),39.2
8,2007-11-09 00:00:00,11,Adelie Penguin (Pygoscelis adeliae),34.1
9,2007-11-09 00:00:00,11,Adelie Penguin (Pygoscelis adeliae),42.0


In [25]:
# Select Month
# Note: equals can be = or ==
#       not equals can be <> or !=
run_sql('''
        SELECT  date_egg as date,
                strftime('%m', date_egg) as month,
                species,
                culmen_length
        FROM penguin
        WHERE month = '12'
        ''', conn_str=con_str)

,date,month,species,culmen_length
0,2007-12-03 00:00:00,12,Gentoo penguin (Pygoscelis papua),45.8
1,2007-12-03 00:00:00,12,Gentoo penguin (Pygoscelis papua),49.3
2,2007-12-03 00:00:00,12,Gentoo penguin (Pygoscelis papua),45.1
3,2007-12-03 00:00:00,12,Gentoo penguin (Pygoscelis papua),59.6
4,2009-12-01 00:00:00,12,Gentoo penguin (Pygoscelis papua),45.5
5,2009-12-01 00:00:00,12,Gentoo penguin (Pygoscelis papua),49.5
6,2009-12-01 00:00:00,12,Gentoo penguin (Pygoscelis papua),47.3
7,2009-12-01 00:00:00,12,Gentoo penguin (Pygoscelis papua),46.8
8,2009-12-01 00:00:00,12,Gentoo penguin (Pygoscelis papua),44.5
9,2009-12-01 00:00:00,12,Gentoo penguin (Pygoscelis papua),48.8


In [26]:
# Select Month
# Note: equals can be = or ==
#       not equals can be <> or !=
run_sql('''
        SELECT  DISTINCT date_egg as date,
                species
        FROM penguin
        ''', conn_str=con_str)

,date,species
0,2007-11-11 00:00:00,Adelie Penguin (Pygoscelis adeliae)
1,2007-11-16 00:00:00,Adelie Penguin (Pygoscelis adeliae)
2,2007-11-15 00:00:00,Adelie Penguin (Pygoscelis adeliae)
3,2007-11-09 00:00:00,Adelie Penguin (Pygoscelis adeliae)
4,2007-11-12 00:00:00,Adelie Penguin (Pygoscelis adeliae)
5,2007-11-10 00:00:00,Adelie Penguin (Pygoscelis adeliae)
6,2007-11-13 00:00:00,Adelie Penguin (Pygoscelis adeliae)
7,2007-11-19 00:00:00,Adelie Penguin (Pygoscelis adeliae)
8,2008-11-06 00:00:00,Adelie Penguin (Pygoscelis adeliae)
9,2008-11-09 00:00:00,Adelie Penguin (Pygoscelis adeliae)


In [27]:
# Select date range
# Note: equals can be = or ==
#       not equals can be <> or !=
run_sql('''
        SELECT  date_egg as date,
                strftime('%m', date_egg) as month,
                species,
                culmen_length
        FROM penguin
        WHERE date BETWEEN '2007-11-11' and '2011-11-11'
        ''', conn_str=con_str)

,date,month,species,culmen_length
0,2007-11-11 00:00:00,11,Adelie Penguin (Pygoscelis adeliae),39.1
1,2007-11-11 00:00:00,11,Adelie Penguin (Pygoscelis adeliae),39.5
2,2007-11-16 00:00:00,11,Adelie Penguin (Pygoscelis adeliae),40.3
3,2007-11-16 00:00:00,11,Adelie Penguin (Pygoscelis adeliae),NaN
4,2007-11-16 00:00:00,11,Adelie Penguin (Pygoscelis adeliae),36.7
5,2007-11-16 00:00:00,11,Adelie Penguin (Pygoscelis adeliae),39.3
6,2007-11-15 00:00:00,11,Adelie Penguin (Pygoscelis adeliae),38.9
7,2007-11-15 00:00:00,11,Adelie Penguin (Pygoscelis adeliae),39.2
8,2007-11-15 00:00:00,11,Adelie Penguin (Pygoscelis adeliae),41.1
9,2007-11-15 00:00:00,11,Adelie Penguin (Pygoscelis adeliae),38.6


In [28]:
# Select date range
# Note: equals can be = or ==
#       not equals can be <> or !=
run_sql('''
        SELECT  date_egg as date,
                strftime('%m', date_egg) as month,
                species,
                culmen_length
        FROM penguin
        WHERE date in ()
        ''', conn_str=con_str)

,date,month,species,culmen_length


In [29]:
# Select date range
# Note: equals can be = or ==
#       not equals can be <> or !=
run_sql('''
        SELECT  date_egg as date,
                strftime('%d', date_egg) as day,
                strftime('%m', date_egg) as month,
                strftime('%Y', date_egg) as year,
                species,
                culmen_length
        FROM penguin
        WHERE day in ('11', '16') AND
              month in ('11', '12')  AND
              year = '2007'
        ''', conn_str=con_str)

,date,day,month,year,species,culmen_length
0,2007-11-11 00:00:00,11,11,2007,Adelie Penguin (Pygoscelis adeliae),39.1
1,2007-11-11 00:00:00,11,11,2007,Adelie Penguin (Pygoscelis adeliae),39.5
2,2007-11-16 00:00:00,16,11,2007,Adelie Penguin (Pygoscelis adeliae),40.3
3,2007-11-16 00:00:00,16,11,2007,Adelie Penguin (Pygoscelis adeliae),NaN
4,2007-11-16 00:00:00,16,11,2007,Adelie Penguin (Pygoscelis adeliae),36.7
5,2007-11-16 00:00:00,16,11,2007,Adelie Penguin (Pygoscelis adeliae),39.3
6,2007-11-16 00:00:00,16,11,2007,Adelie Penguin (Pygoscelis adeliae),34.6
7,2007-11-16 00:00:00,16,11,2007,Adelie Penguin (Pygoscelis adeliae),36.6
8,2007-11-16 00:00:00,16,11,2007,Adelie Penguin (Pygoscelis adeliae),34.4
9,2007-11-16 00:00:00,16,11,2007,Adelie Penguin (Pygoscelis adeliae),46.0


In [30]:
# Select date range
# Note: equals can be = or ==
#       not equals can be <> or !=
run_sql('''
        SELECT  date_egg as date,
                strftime('%d', date_egg) as day,
                strftime('%m', date_egg) as month,
                strftime('%Y', date_egg) as year,
                species,
                culmen_length
        FROM penguin
        WHERE strftime('%d-%m-%Y', date_egg) IN ('11-11-2007', '16-11-2007')
        ''', conn_str=con_str)

,date,day,month,year,species,culmen_length
0,2007-11-11 00:00:00,11,11,2007,Adelie Penguin (Pygoscelis adeliae),39.1
1,2007-11-11 00:00:00,11,11,2007,Adelie Penguin (Pygoscelis adeliae),39.5
2,2007-11-16 00:00:00,16,11,2007,Adelie Penguin (Pygoscelis adeliae),40.3
3,2007-11-16 00:00:00,16,11,2007,Adelie Penguin (Pygoscelis adeliae),NaN
4,2007-11-16 00:00:00,16,11,2007,Adelie Penguin (Pygoscelis adeliae),36.7
5,2007-11-16 00:00:00,16,11,2007,Adelie Penguin (Pygoscelis adeliae),39.3
6,2007-11-16 00:00:00,16,11,2007,Adelie Penguin (Pygoscelis adeliae),34.6
7,2007-11-16 00:00:00,16,11,2007,Adelie Penguin (Pygoscelis adeliae),36.6
8,2007-11-16 00:00:00,16,11,2007,Adelie Penguin (Pygoscelis adeliae),34.4
9,2007-11-16 00:00:00,16,11,2007,Adelie Penguin (Pygoscelis adeliae),46.0


In [31]:
# Select Month
# Note: equals can be = or ==
#       not equals can be <> or !=
# Casting - (expression not a function)
run_sql('''
        SELECT  date_egg as date,
                CAST(STRFTIME('%m', date_egg) as INTEGER) as month,
                CAST(STRFTIME('%Y', date_egg) as INTEGER) as year,
                species,
                culmen_length
        FROM penguin
        WHERE (month = 12 and year = 2009) OR (month = 11 and year = 2007)
        ''', conn_str=con_str)

,date,month,year,species,culmen_length
0,2007-11-11 00:00:00,11,2007,Adelie Penguin (Pygoscelis adeliae),39.1
1,2007-11-11 00:00:00,11,2007,Adelie Penguin (Pygoscelis adeliae),39.5
2,2007-11-16 00:00:00,11,2007,Adelie Penguin (Pygoscelis adeliae),40.3
3,2007-11-16 00:00:00,11,2007,Adelie Penguin (Pygoscelis adeliae),NaN
4,2007-11-16 00:00:00,11,2007,Adelie Penguin (Pygoscelis adeliae),36.7
5,2007-11-16 00:00:00,11,2007,Adelie Penguin (Pygoscelis adeliae),39.3
6,2007-11-15 00:00:00,11,2007,Adelie Penguin (Pygoscelis adeliae),38.9
7,2007-11-15 00:00:00,11,2007,Adelie Penguin (Pygoscelis adeliae),39.2
8,2007-11-09 00:00:00,11,2007,Adelie Penguin (Pygoscelis adeliae),34.1
9,2007-11-09 00:00:00,11,2007,Adelie Penguin (Pygoscelis adeliae),42.0


In [32]:
# Select Month
# Note: equals can be = or ==
#       not equals can be <> or !=
# Casting - (expression not a function)
# What if I do not want to show month and year - USE Common Table Expressions or CTEs - (or use Pandas)

run_sql('''
        WITH INTERMEDIATE AS (
                                SELECT  date_egg as date,
                                        CAST(STRFTIME('%m', date_egg) as INTEGER) as month,
                                        CAST(STRFTIME('%Y', date_egg) as INTEGER) as year,
                                        species,
                                        culmen_length
                                FROM penguin
                                WHERE (month = 12 and year = 2009) OR (month = 11 and year = 2007)
                              )
        
        SELECT date,
               species,
               culmen_length
        FROM INTERMEDIATE       
        ''', conn_str=con_str)

,date,species,culmen_length
0,2007-11-11 00:00:00,Adelie Penguin (Pygoscelis adeliae),39.1
1,2007-11-11 00:00:00,Adelie Penguin (Pygoscelis adeliae),39.5
2,2007-11-16 00:00:00,Adelie Penguin (Pygoscelis adeliae),40.3
3,2007-11-16 00:00:00,Adelie Penguin (Pygoscelis adeliae),NaN
4,2007-11-16 00:00:00,Adelie Penguin (Pygoscelis adeliae),36.7
5,2007-11-16 00:00:00,Adelie Penguin (Pygoscelis adeliae),39.3
6,2007-11-15 00:00:00,Adelie Penguin (Pygoscelis adeliae),38.9
7,2007-11-15 00:00:00,Adelie Penguin (Pygoscelis adeliae),39.2
8,2007-11-09 00:00:00,Adelie Penguin (Pygoscelis adeliae),34.1
9,2007-11-09 00:00:00,Adelie Penguin (Pygoscelis adeliae),42.0


## Working with Strings

Please note the ff:
- SQL is case insensitive.
- % matches 0 or more whilst _ matches 1 or more characters. 

In [33]:
# Select rows/records where individual ID following 'N' has 2 DIGITS

run_sql('''
        SELECT *
        FROM penguin
        WHERE individual_id LIKE 'N__A%'
        ''',
        conn_str=con_str
)

,index,studyname,sample_number,species,region,island,stage,individual_id,clutch_completion,date_egg,culmen_length,culmen_depth,flipper_length,body_mass,sex,delta_15_n,delta_13_c,comments
0,18,PAL0708,19,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N10A1,Yes,2007-11-16 00:00:00,34.4,18.4,184.0,3325.0,FEMALE,8.47827,-25.23319,None
1,19,PAL0708,20,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N10A2,Yes,2007-11-16 00:00:00,46.0,21.5,194.0,4200.0,MALE,9.11616,-24.77227,None
2,20,PAL0708,21,Adelie Penguin (Pygoscelis adeliae),Anvers,Biscoe,"Adult, 1 Egg Stage",N11A1,Yes,2007-11-12 00:00:00,37.8,18.3,174.0,3400.0,FEMALE,8.73762,-25.09383,None
3,21,PAL0708,22,Adelie Penguin (Pygoscelis adeliae),Anvers,Biscoe,"Adult, 1 Egg Stage",N11A2,Yes,2007-11-12 00:00:00,37.7,18.7,180.0,3600.0,MALE,8.66271,-25.06390,None
4,22,PAL0708,23,Adelie Penguin (Pygoscelis adeliae),Anvers,Biscoe,"Adult, 1 Egg Stage",N12A1,Yes,2007-11-12 00:00:00,35.9,19.2,189.0,3800.0,FEMALE,9.22286,-25.03474,None
5,23,PAL0708,24,Adelie Penguin (Pygoscelis adeliae),Anvers,Biscoe,"Adult, 1 Egg Stage",N12A2,Yes,2007-11-12 00:00:00,38.2,18.1,185.0,3950.0,MALE,8.43423,-25.22664,None
6,24,PAL0708,25,Adelie Penguin (Pygoscelis adeliae),Anvers,Biscoe,"Adult, 1 Egg Stage",N13A1,Yes,2007-11-10 00:00:00,38.8,17.2,180.0,3800.0,MALE,9.63954,-25.29856,None
7,25,PAL0708,26,Adelie Penguin (Pygoscelis adeliae),Anvers,Biscoe,"Adult, 1 Egg Stage",N13A2,Yes,2007-11-10 00:00:00,35.3,18.9,187.0,3800.0,FEMALE,9.21292,-24.36130,None
8,26,PAL0708,27,Adelie Penguin (Pygoscelis adeliae),Anvers,Biscoe,"Adult, 1 Egg Stage",N17A1,Yes,2007-11-12 00:00:00,40.6,18.6,183.0,3550.0,MALE,8.93997,-25.36288,None
9,27,PAL0708,28,Adelie Penguin (Pygoscelis adeliae),Anvers,Biscoe,"Adult, 1 Egg Stage",N17A2,Yes,2007-11-12 00:00:00,40.5,17.9,187.0,3200.0,FEMALE,8.08138,-25.49448,None


In [37]:
# Select all females from Torgesen island, order by date descending

run_sql('''
        SELECT *
        FROM penguin
        WHERE island = 'Torgersen' AND sex = 'FEMALE'
        ORDER BY date_egg DESC
        ''',
        conn_str=con_str
)

,index,studyname,sample_number,species,region,island,stage,individual_id,clutch_completion,date_egg,culmen_length,culmen_depth,flipper_length,body_mass,sex,delta_15_n,delta_13_c,comments
0,130,PAL0910,131,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N73A1,No,2009-11-23 00:00:00,38.5,17.9,190.0,3325.0,FEMALE,8.98460,-25.57956,Nest never observed with full clutch.
1,118,PAL0910,119,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N64A1,Yes,2009-11-22 00:00:00,35.7,17.0,189.0,3350.0,FEMALE,8.96436,-23.90309,None
2,126,PAL0910,127,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N71A1,Yes,2009-11-21 00:00:00,38.8,17.6,191.0,3275.0,FEMALE,8.88098,-25.89741,None
3,116,PAL0910,117,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N63A1,Yes,2009-11-18 00:00:00,38.6,17.0,188.0,2900.0,FEMALE,9.18021,-25.77264,None
4,124,PAL0910,125,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N69A1,Yes,2009-11-18 00:00:00,35.2,15.9,186.0,3050.0,FEMALE,8.81668,-25.95399,None
5,128,PAL0910,129,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N72A1,Yes,2009-11-18 00:00:00,39.0,17.1,191.0,3050.0,FEMALE,9.19031,-25.73722,None
6,120,PAL0910,121,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N66A1,No,2009-11-17 00:00:00,36.2,17.2,187.0,3150.0,FEMALE,9.04296,-26.19444,Nest never observed with full clutch.
7,122,PAL0910,123,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N67A1,Yes,2009-11-16 00:00:00,40.2,17.0,176.0,3450.0,FEMALE,9.30722,-25.61039,None
8,70,PAL0809,71,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N34A1,Yes,2008-11-14 00:00:00,33.5,19.0,190.0,3600.0,FEMALE,7.88863,-26.63085,None
9,68,PAL0809,69,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N32A1,No,2008-11-11 00:00:00,35.9,16.6,190.0,3050.0,FEMALE,8.47781,-26.07821,Nest never observed with full clutch.


In [38]:
#  Select the record where species contains pygo
run_sql('''
        SELECT *
        FROM penguin
        WHERE species LIKE '%pygo%'
        ORDER BY date_egg DESC
        ''',
        conn_str=con_str
)

,index,studyname,sample_number,species,region,island,stage,individual_id,clutch_completion,date_egg,culmen_length,culmen_depth,flipper_length,body_mass,sex,delta_15_n,delta_13_c,comments
0,244,PAL0910,93,Gentoo penguin (Pygoscelis papua),Anvers,Biscoe,"Adult, 1 Egg Stage",N18A1,Yes,2009-12-01 00:00:00,45.5,14.5,212.0,4750.0,FEMALE,8.15566,-26.22848,None
1,245,PAL0910,94,Gentoo penguin (Pygoscelis papua),Anvers,Biscoe,"Adult, 1 Egg Stage",N18A2,Yes,2009-12-01 00:00:00,49.5,16.1,224.0,5650.0,MALE,8.83352,-25.69195,None
2,256,PAL0910,105,Gentoo penguin (Pygoscelis papua),Anvers,Biscoe,"Adult, 1 Egg Stage",N24A1,Yes,2009-12-01 00:00:00,47.3,13.8,216.0,4725.0,None,8.25818,-26.23886,Sexing primers did not amplify.
3,257,PAL0910,106,Gentoo penguin (Pygoscelis papua),Anvers,Biscoe,"Adult, 1 Egg Stage",N24A2,Yes,2009-12-01 00:00:00,46.8,16.1,215.0,5500.0,MALE,8.32359,-26.05756,None
4,268,PAL0910,117,Gentoo penguin (Pygoscelis papua),Anvers,Biscoe,"Adult, 1 Egg Stage",N36A1,Yes,2009-12-01 00:00:00,44.5,15.7,217.0,4875.0,None,8.04111,-26.18444,Sexing primers did not amplify.
5,269,PAL0910,118,Gentoo penguin (Pygoscelis papua),Anvers,Biscoe,"Adult, 1 Egg Stage",N36A2,Yes,2009-12-01 00:00:00,48.8,16.2,222.0,6000.0,MALE,8.33825,-25.88547,None
6,270,PAL0910,119,Gentoo penguin (Pygoscelis papua),Anvers,Biscoe,"Adult, 1 Egg Stage",N38A1,No,2009-12-01 00:00:00,47.2,13.7,214.0,4925.0,FEMALE,7.99184,-26.20538,Nest never observed with full clutch.
7,271,PAL0910,120,Gentoo penguin (Pygoscelis papua),Anvers,Biscoe,"Adult, 1 Egg Stage",N38A2,No,2009-12-01 00:00:00,NaN,NaN,NaN,NaN,None,NaN,NaN,Adult not sampled. Nest never observed with fu...
8,246,PAL0910,95,Gentoo penguin (Pygoscelis papua),Anvers,Biscoe,"Adult, 1 Egg Stage",N19A1,Yes,2009-11-27 00:00:00,44.5,14.7,214.0,4850.0,FEMALE,8.20106,-26.16524,None
9,247,PAL0910,96,Gentoo penguin (Pygoscelis papua),Anvers,Biscoe,"Adult, 1 Egg Stage",N19A2,Yes,2009-11-27 00:00:00,50.8,15.7,226.0,5200.0,MALE,8.27102,-26.11244,None


In [39]:
#  Select the distinct island names from studyname PAL0910
run_sql('''
        SELECT DISTINCT island
        FROM penguin
        WHERE studyname = 'PAL0910'
        ''',
        conn_str=con_str
)

,island
0,Biscoe
1,Torgersen
2,Dream


In [41]:
#  Select the distinct REGIONS for Adelie species
run_sql('''
        SELECT DISTINCT region
        FROM penguin
        WHERE species LIKE '%adelie%'
        ''',
        conn_str=con_str
)

,region
0,Anvers


## Working with NULLS

In [42]:
# Showing culmen_length Null Records

run_sql('''
        SELECT *
        FROM penguin
        WHERE culmen_length IS NULL        
        ''',
        conn_str=con_str
        )

,index,studyname,sample_number,species,region,island,stage,individual_id,clutch_completion,date_egg,culmen_length,culmen_depth,flipper_length,body_mass,sex,delta_15_n,delta_13_c,comments
0,3,PAL0708,4,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N2A2,Yes,2007-11-16 00:00:00,None,None,None,None,None,None,None,Adult not sampled.
1,271,PAL0910,120,Gentoo penguin (Pygoscelis papua),Anvers,Biscoe,"Adult, 1 Egg Stage",N38A2,No,2009-12-01 00:00:00,None,None,None,None,None,None,None,Adult not sampled. Nest never observed with fu...


In [45]:
# Showing culmen_length Records NOT NULL

run_sql('''
        SELECT *
        FROM penguin
        WHERE culmen_length IS NOT NULL        
        ''',
        conn_str=con_str
        )

,index,studyname,sample_number,species,region,island,stage,individual_id,clutch_completion,date_egg,culmen_length,culmen_depth,flipper_length,body_mass,sex,delta_15_n,delta_13_c,comments
0,0,PAL0708,1,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A1,Yes,2007-11-11 00:00:00,39.1,18.7,181.0,3750.0,MALE,NaN,NaN,Not enough blood for isotopes.
1,1,PAL0708,2,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A2,Yes,2007-11-11 00:00:00,39.5,17.4,186.0,3800.0,FEMALE,8.94956,-24.69454,None
2,2,PAL0708,3,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N2A1,Yes,2007-11-16 00:00:00,40.3,18.0,195.0,3250.0,FEMALE,8.36821,-25.33302,None
3,4,PAL0708,5,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N3A1,Yes,2007-11-16 00:00:00,36.7,19.3,193.0,3450.0,FEMALE,8.76651,-25.32426,None
4,5,PAL0708,6,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N3A2,Yes,2007-11-16 00:00:00,39.3,20.6,190.0,3650.0,MALE,8.66496,-25.29805,None
5,6,PAL0708,7,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N4A1,No,2007-11-15 00:00:00,38.9,17.8,181.0,3625.0,FEMALE,9.18718,-25.21799,Nest never observed with full clutch.
6,7,PAL0708,8,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N4A2,No,2007-11-15 00:00:00,39.2,19.6,195.0,4675.0,MALE,9.46060,-24.89958,Nest never observed with full clutch.
7,8,PAL0708,9,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N5A1,Yes,2007-11-09 00:00:00,34.1,18.1,193.0,3475.0,None,NaN,NaN,No blood sample obtained.
8,9,PAL0708,10,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N5A2,Yes,2007-11-09 00:00:00,42.0,20.2,190.0,4250.0,None,9.13362,-25.09368,No blood sample obtained for sexing.
9,10,PAL0708,11,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N6A1,Yes,2007-11-09 00:00:00,37.8,17.1,186.0,3300.0,None,8.63243,-25.21315,No blood sample obtained for sexing.


In [44]:
# Showing Null Records

run_sql('''
        SELECT  culmen_length,
                culmen_depth,
                flipper_length,
                COALESCE(culmen_length/culmen_depth, -1) as ratio,
                body_mass
        FROM penguin     
        ''',
        conn_str=con_str
        )

,culmen_length,culmen_depth,flipper_length,ratio,body_mass
0,39.1,18.7,181.0,2.090909,3750.0
1,39.5,17.4,186.0,2.270115,3800.0
2,40.3,18.0,195.0,2.238889,3250.0
3,NaN,NaN,NaN,-1.000000,NaN
4,36.7,19.3,193.0,1.901554,3450.0
5,39.3,20.6,190.0,1.907767,3650.0
6,38.9,17.8,181.0,2.185393,3625.0
7,39.2,19.6,195.0,2.000000,4675.0
8,34.1,18.1,193.0,1.883978,3475.0
9,42.0,20.2,190.0,2.079208,4250.0


## GROUP BY Aggregations

In [48]:
run_sql('''
        SELECT species,
               culmen_length,
               culmen_depth  
        FROM penguin
        ''',
        conn_str=con_str
)

,species,culmen_length,culmen_depth
0,Adelie Penguin (Pygoscelis adeliae),39.1,18.7
1,Adelie Penguin (Pygoscelis adeliae),39.5,17.4
2,Adelie Penguin (Pygoscelis adeliae),40.3,18.0
3,Adelie Penguin (Pygoscelis adeliae),NaN,NaN
4,Adelie Penguin (Pygoscelis adeliae),36.7,19.3
5,Adelie Penguin (Pygoscelis adeliae),39.3,20.6
6,Adelie Penguin (Pygoscelis adeliae),38.9,17.8
7,Adelie Penguin (Pygoscelis adeliae),39.2,19.6
8,Adelie Penguin (Pygoscelis adeliae),34.1,18.1
9,Adelie Penguin (Pygoscelis adeliae),42.0,20.2


In [49]:
# Average culmen_length and culmen_depth (Not IDEAL)
run_sql('''
        SELECT species,
               avg(culmen_length),
               avg(culmen_depth)  
        FROM penguin
        ''',
        conn_str=con_str
)

,species,avg(culmen_length),avg(culmen_depth)
0,Adelie Penguin (Pygoscelis adeliae),43.92193,17.15117


In [50]:
# Average culmen_length and culmen_depth per group
run_sql('''
        SELECT species,
               avg(culmen_length),
               avg(culmen_depth)  
        FROM penguin
        GROUP BY species
        ''',
        conn_str=con_str
)

,species,avg(culmen_length),avg(culmen_depth)
0,Adelie Penguin (Pygoscelis adeliae),38.791391,18.346358
1,Chinstrap penguin (Pygoscelis antarctica),48.833824,18.420588
2,Gentoo penguin (Pygoscelis papua),47.504878,14.982114


In [51]:
# Average culmen_length and culmen_depth per species per year
run_sql('''
        SELECT species,
               strftime('%Y', date_egg)  as year,
               avg(culmen_length),
               avg(culmen_depth)  
        FROM penguin
        GROUP BY species, year
        ''',
        conn_str=con_str
)

,species,year,avg(culmen_length),avg(culmen_depth)
0,Adelie Penguin (Pygoscelis adeliae),2007,38.824490,18.767347
1,Adelie Penguin (Pygoscelis adeliae),2008,38.560000,18.192000
2,Adelie Penguin (Pygoscelis adeliae),2009,38.982692,18.098077
3,Chinstrap penguin (Pygoscelis antarctica),2007,48.723077,18.484615
4,Chinstrap penguin (Pygoscelis antarctica),2008,48.700000,18.450000
5,Chinstrap penguin (Pygoscelis antarctica),2009,49.054167,18.329167
6,Gentoo penguin (Pygoscelis papua),2007,47.014706,14.688235
7,Gentoo penguin (Pygoscelis papua),2008,46.936957,14.923913
8,Gentoo penguin (Pygoscelis papua),2009,48.500000,15.276744


In [54]:
# Select the count of sex per year

run_sql('''
        SELECT  strftime('%Y', date_egg)  AS year,
                count(*) AS count
        FROM penguin
        GROUP BY year
        ''',
        conn_str=con_str
        )

,year,count
0,2007,110
1,2008,114
2,2009,120


In [55]:
# Select the count of sex 

run_sql('''
        SELECT  sex,
                count(*) AS count
        FROM penguin
        GROUP BY sex
        ''',
        conn_str=con_str
        )

,sex,count
0,None,11
1,FEMALE,165
2,MALE,168


In [56]:
# Select the count of species 

run_sql('''
        SELECT  species,
                count(*) AS count
        FROM penguin
        GROUP BY species
        ''',
        conn_str=con_str
        )

,species,count
0,Adelie Penguin (Pygoscelis adeliae),152
1,Chinstrap penguin (Pygoscelis antarctica),68
2,Gentoo penguin (Pygoscelis papua),124


In [58]:
# Select the count of each species on each Island

run_sql('''
        SELECT species,
               island,
               count(*) as count
        FROM penguin
        GROUP BY  species, island
        ORDER BY count DESC
        ''',
        conn_str=con_str
        )

,species,island,count
0,Gentoo penguin (Pygoscelis papua),Biscoe,124
1,Chinstrap penguin (Pygoscelis antarctica),Dream,68
2,Adelie Penguin (Pygoscelis adeliae),Dream,56
3,Adelie Penguin (Pygoscelis adeliae),Torgersen,52
4,Adelie Penguin (Pygoscelis adeliae),Biscoe,44


In [60]:
# Select the average mass by species and sex

run_sql('''
        SELECT species,
               sex,
               avg(body_mass) 
        FROM penguin
        GROUP BY species, sex
        ''', 
        conn_str=con_str
    )

,species,sex,avg(body_mass)
0,Adelie Penguin (Pygoscelis adeliae),None,3540.000000
1,Adelie Penguin (Pygoscelis adeliae),FEMALE,3368.835616
2,Adelie Penguin (Pygoscelis adeliae),MALE,4043.493151
3,Chinstrap penguin (Pygoscelis antarctica),FEMALE,3527.205882
4,Chinstrap penguin (Pygoscelis antarctica),MALE,3938.970588
5,Gentoo penguin (Pygoscelis papua),None,4587.500000
6,Gentoo penguin (Pygoscelis papua),FEMALE,4679.741379
7,Gentoo penguin (Pygoscelis papua),MALE,5484.836066


In [61]:
# Select the average mass by species and sex

run_sql('''
        SELECT species,
               sex,
               min(body_mass),
               max(body_mass) 
        FROM penguin
        GROUP BY species, sex
        ''', 
        conn_str=con_str
    )

,species,sex,min(body_mass),max(body_mass)
0,Adelie Penguin (Pygoscelis adeliae),None,2975.0,4250.0
1,Adelie Penguin (Pygoscelis adeliae),FEMALE,2850.0,3900.0
2,Adelie Penguin (Pygoscelis adeliae),MALE,3325.0,4775.0
3,Chinstrap penguin (Pygoscelis antarctica),FEMALE,2700.0,4150.0
4,Chinstrap penguin (Pygoscelis antarctica),MALE,3250.0,4800.0
5,Gentoo penguin (Pygoscelis papua),None,4100.0,4875.0
6,Gentoo penguin (Pygoscelis papua),FEMALE,3950.0,5200.0
7,Gentoo penguin (Pygoscelis papua),MALE,4750.0,6300.0


In [70]:
run_sql('''
        SELECT  species,
                CASE WHEN body_mass > 5000
                          THEN 'L'
                     WHEN body_mass < 4000
                          THEN 'S'
                     ELSE 'M' END as size,
                count(*)
        FROM penguin
        GROUP BY species, size
        ORDER BY species ASC, size DESC
        ''',
        conn_str=con_str
        )

,species,size,count(*)
0,Adelie Penguin (Pygoscelis adeliae),S,112
1,Adelie Penguin (Pygoscelis adeliae),M,40
2,Chinstrap penguin (Pygoscelis antarctica),S,52
3,Chinstrap penguin (Pygoscelis antarctica),M,16
4,Gentoo penguin (Pygoscelis papua),S,1
5,Gentoo penguin (Pygoscelis papua),M,62
6,Gentoo penguin (Pygoscelis papua),L,61


## Sorting

In [71]:
# Select the count of each species on each Island

run_sql('''
        SELECT species,
               island,
               count(*) as count
        FROM penguin
        GROUP BY  species, island
        ORDER BY count DESC, species ASC
        ''',
        conn_str=con_str
        )

,species,island,count
0,Gentoo penguin (Pygoscelis papua),Biscoe,124
1,Chinstrap penguin (Pygoscelis antarctica),Dream,68
2,Adelie Penguin (Pygoscelis adeliae),Dream,56
3,Adelie Penguin (Pygoscelis adeliae),Torgersen,52
4,Adelie Penguin (Pygoscelis adeliae),Biscoe,44


## Filtering Aggregations

- Pre-filter with `WHERE`
- Post filter with `HAVING`
- Use HAVING to filter aggregations

In [73]:
run_sql('''
        SELECT species,
               sex,
               avg(body_mass) as avg_body
        FROM penguin
        GROUP BY species, sex
        HAVING avg_body > 3500
        ''', 
        conn_str=con_str
    )

,species,sex,avg_body
0,Adelie Penguin (Pygoscelis adeliae),None,3540.000000
1,Adelie Penguin (Pygoscelis adeliae),MALE,4043.493151
2,Chinstrap penguin (Pygoscelis antarctica),FEMALE,3527.205882
3,Chinstrap penguin (Pygoscelis antarctica),MALE,3938.970588
4,Gentoo penguin (Pygoscelis papua),None,4587.500000
5,Gentoo penguin (Pygoscelis papua),FEMALE,4679.741379
6,Gentoo penguin (Pygoscelis papua),MALE,5484.836066


In [76]:
# Select the maximum mass for each region and island for Adelie penguin. Sort by region and island in descending

run_sql('''
        SELECT region,
                island,
                max(body_mass)
        FROM penguin
        WHERE species LIKE '%Adelie%'
        GROUP BY region, island 
        ORDER BY region, island DESC
        ''',
        conn_str=con_str
        )

,region,island,max(body_mass)
0,Anvers,Torgersen,4700.0
1,Anvers,Dream,4650.0
2,Anvers,Biscoe,4775.0


In [81]:
# Select the average mass and the maximum mass for each species. Sort by average mass descending. Only include groups with average less than 4000


run_sql('''
        SELECT species,
                avg(body_mass) AS avg_mass,
                max(body_mass) AS max_mass
        FROM penguin
        GROUP BY species
        HAVING avg_mass < 4000
        ORDER BY avg_mass DESC
        ''',
        conn_str=con_str)

,species,avg_mass,max_mass
0,Chinstrap penguin (Pygoscelis antarctica),3733.088235,4800.0
1,Adelie Penguin (Pygoscelis adeliae),3700.662252,4775.0


In [85]:
# Select the count for each study and island. Only include the those that have a count of more than 25

run_sql('''
        SELECT studyname,
                island,
                count(*) as count
        FROM penguin
        GROUP BY studyname, island
        HAVING count > 25
        ''',
        conn_str=con_str
        )

,studyname,island,count
0,PAL0708,Biscoe,44
1,PAL0708,Dream,46
2,PAL0809,Biscoe,64
3,PAL0809,Dream,34
4,PAL0910,Biscoe,60
5,PAL0910,Dream,44


## Query Parameters

> Use query parameters instead of string concatenations to receive user input when running queries or exposing your database.

In [94]:
# Create a query count that summarizes penguin count and avg mass by island and sex. Insert a place holder to have a where statement that filters by island

run_sql('''
        SELECT  island,
                sex,
                count(*) as penguin_count,
                avg(body_mass) as avg_ass
        FROM penguin
        WHERE island = ?
        GROUP BY island, sex
        ''',
        conn_str=con_str, values=('Torgersen',))

,island,sex,penguin_count,avg_ass
0,Torgersen,None,5,3681.250000
1,Torgersen,FEMALE,24,3395.833333
2,Torgersen,MALE,23,4034.782609
